In [1]:
import os
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix, vstack, save_npz, load_npz

import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
from tqdm.notebook import tqdm
from sys import getsizeof
import gc
#from catboost import CatBoostRegressor, cv, Pool, sum_models
from sklearn.model_selection import StratifiedKFold, train_test_split
#from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import QuantileTransformer, MaxAbsScaler

import vaex
import pyarrow.parquet as pq
import bisect

import pickle
from random import shuffle
import datetime

C:\ProgramData\Miniconda3\envs\ml_env_v1\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Miniconda3\envs\ml_env_v1\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Miniconda3\envs\ml_env_v1\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
LOCAL_DATA_PATH = 'context_data'
SPLIT_SEED = 42
DATA_FILE = 'competition_data_final_pqt'
TARGET_FILE = 'public_train.pqt'
SUBMISSION_FILE = 'submit_2.pqt'

In [4]:
df = vaex.open(f'../{LOCAL_DATA_PATH}/{DATA_FILE}')

In [5]:
%%time

df['date'] = df.date.astype(str)
dates_list = df.date.unique()
dow_dict = {}
for dtx in dates_list:
    dow_dict[dtx] = datetime.datetime.strptime(dtx, '%Y-%m-%d').strftime("%A")
df['dow'] = df['date'].map(dow_dict)

CPU times: total: 3min 12s
Wall time: 14.1 s


In [6]:
data_agg_pod = df[['user_id', 'url_host', 'part_of_day', 'request_cnt']].\
    groupby(['user_id', 'url_host', 'part_of_day']).agg([('request_cnt', "sum")])
data_agg_pod['url_host_pod'] = data_agg_pod.url_host + '_' + data_agg_pod.part_of_day

In [7]:
url_list = data_agg_pod['url_host_pod'].unique()
url_dict = {url: idurl for url, idurl in zip(url_list, range(len(url_list)))}
print(f'{len(url_dict)} urls')

474010 urls


In [8]:
values = data_agg_pod['request_cnt'].values.astype(np.uint16)
rows = data_agg_pod['user_id'].to_numpy().astype(np.int32)
cols = data_agg_pod['url_host_pod'].map(url_dict).to_numpy().astype(np.int32)
mat = csr_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
mat

<415317x474010 sparse matrix of type '<class 'numpy.uint16'>'
	with 63003895 stored elements in Compressed Sparse Row format>

In [9]:
save_npz('mat_pod.npz', mat)